# RISE Camp Code Snippet

In [1]:
# TODO: Filter out any imports that are not necessary
import ray
ray.init(ignore_reinit_error=True, num_cpus=32, _temp_dir="/home/brian/external/aws-asdi/ray_temp"); # ray.init() config for single node setup
#ray.init(ignore_reinit_error=True, address="auto", _redis_password='5241590000000000'); # ray.init() config for cluster setup
import modin.pandas as pd
from nums import numpy as nps

from nums.core import settings
from nums.core import application_manager
from nums.experimental.nums_modin import from_modin
nps_app_inst = application_manager.instance()

from tqdm.auto import tqdm
import os
import warnings
warnings.filterwarnings("ignore");

2021-10-28 15:47:44,451	INFO services.py:1265 -- View the Ray dashboard at http://127.0.0.1:8266


Using driver node ip as head node.
head node 10.0.0.4
total cpus 32.0
device_grid (0, 0) 0=node:10.0.0.4/cpu:1


In [2]:
def rmse(actual, expected):
    """
    Computes the root mean squared error to evaluate models/predictions. It can accept all the datatypes used
    in this notebook
    """
    import numpy as np
    if type(actual) != type(expected):
        raise TypeError("actual and expected must be the same types")
    elif type(actual) == type(nps.array([])):
        return nps.sqrt(nps.mean((expected - actual) ** 2)).get()
    elif type(actual) == type(np.array([])):
        return np.sqrt(np.mean((expected - actual) ** 2))
    

In [3]:
# NumS cluster setting
if len(ray.nodes()) > 1:
    settings.cluster_shape = (len(ray.nodes())-1, 1)
settings.cluster_shape

(1, 1)

In [4]:
data = pd.DataFrame()

In [5]:
# for year in tqdm(list(range(2000, 2022))):
#     X = pd.read_csv("data/" + str(year) + ".csv")
#     data = data.append(X)

data = data.dropna()

In [6]:
# features = ['YEAR/MONTH/DAY', 'TMAX', 'TMIN', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'TAVG', 'TRANGE']
# labels = ['PRCP']

In [7]:
# X = from_modin(data[features]).astype(float)
# y = from_modin(data[labels]).astype(float).reshape(-1)

In [8]:
X = from_modin(pd.read_csv("data/X_2020.csv")).astype(float)
y = from_modin(pd.read_csv("data/y_2020.csv")).astype(float).reshape(-1)

In [9]:
split = int(X.shape[0] * 0.8)
X_train = X[:split]
X_test = X[split:]
y_train = y[:split]
y_test = y[split:]

In [10]:
from nums.models.glms import LinearRegression

model = LinearRegression(solver='sgd')

In [11]:
%%time
model.fit(X_train, y_train)

CPU times: user 171 ms, sys: 24.6 ms, total: 195 ms
Wall time: 98.8 ms


In [12]:
%%time
training_results = model.predict(X_train)

CPU times: user 38.5 ms, sys: 9.6 ms, total: 48.1 ms
Wall time: 15.8 ms


In [13]:
%%time
test_results = model.predict(X_test)

CPU times: user 10.6 ms, sys: 506 µs, total: 11.1 ms
Wall time: 5.11 ms


In [14]:
print("Training RMSE:", rmse(training_results, y_train))
print("Testing RMSE", rmse(test_results, y_test))

Training RMSE: 82.30906549322629
Testing RMSE 78.66980266376879


In [15]:
split = int(X.shape[0] * 0.8)
X_train_np = X_train.get()
X_test_np = X_test.get()
y_train_np = y_train.get()
y_test_np = y_test.get()

In [16]:
from sklearn.linear_model import SGDRegressor

model = SGDRegressor(penalty='l2', max_iter=100, tol=0.0001)

In [17]:
%%time
model.fit(X_train_np, y_train_np)

CPU times: user 1min 5s, sys: 359 ms, total: 1min 5s
Wall time: 1min 3s


SGDRegressor(max_iter=100, tol=0.0001)

In [18]:
%%time
training_results = model.predict(X_train_np)

CPU times: user 46.6 ms, sys: 3.96 ms, total: 50.6 ms
Wall time: 46.9 ms


In [19]:
%%time
test_results = model.predict(X_test_np)

CPU times: user 11.8 ms, sys: 16.2 ms, total: 28 ms
Wall time: 25.9 ms


In [20]:
print("Training RMSE:", rmse(training_results, y_train_np))
print("Testing RMSE", rmse(test_results, y_test_np))

Training RMSE: 120269903936624.47
Testing RMSE 203209187595299.28


In [21]:
X = from_modin(pd.read_csv("data/X_2020.csv")).astype(float)
y = from_modin(pd.read_csv("data/y_bin_2020.csv")).astype(float).reshape(-1)

In [22]:
split = int(X.shape[0] * 0.8)
X_train = X[:split]
X_test = X[split:]
y_train = y[:split]
y_test = y[split:]

In [23]:
from nums.models.glms import LogisticRegression

model = LogisticRegression(solver="newton", penalty="l2", tol=1e-4)

In [24]:
%%time
model.fit(X_train, y_train)

CPU times: user 2.81 s, sys: 723 ms, total: 3.54 s
Wall time: 1.38 s


In [25]:
%%time
training_results = model.predict(X_train)

CPU times: user 142 ms, sys: 29.1 ms, total: 171 ms
Wall time: 66.1 ms


In [26]:
%%time
test_results = model.predict(X_test)

CPU times: user 42.2 ms, sys: 8.33 ms, total: 50.5 ms
Wall time: 20 ms


In [27]:
print("Training accuracy: ", nps.sum(training_results == y_train).get() / y_train.shape[0])
print("Testing accuracy:  ", nps.sum(test_results == y_test).get() / y_test.shape[0])

Training accuracy:  0.7166651577539586
Testing accuracy:   0.7431014065353085


In [28]:
split = int(X.shape[0] * 0.8)
X_train_np = X_train.get()
X_test_np = X_test.get()
y_train_np = y_train.get()
y_test_np = y_test.get()

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
model = LogisticRegression()

In [31]:
%%time
model.fit(X_train_np, y_train_np)

CPU times: user 1min 23s, sys: 17.7 s, total: 1min 41s
Wall time: 1min 39s


LogisticRegression()

In [32]:
%%time
training_results = model.predict(X_train_np)

CPU times: user 111 ms, sys: 56.4 ms, total: 168 ms
Wall time: 159 ms


In [33]:
%%time
test_results = model.predict(X_test_np)

CPU times: user 37.3 ms, sys: 4.16 ms, total: 41.5 ms
Wall time: 39 ms


In [34]:
print("Training accuracy: ", model.score(X_train_np, y_train_np))
print("Testing accuracy:  ", model.score(X_test_np, y_test_np))

Training accuracy:  0.7085399493234532
Testing accuracy:   0.7356080342885957
